In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install pytorch-metric-learning

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.1/119.1 kB 3.1 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-m

In [ ]:
import pandas as pd
import os
import cv2
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader
from pytorch_metric_learning import distances, losses, miners, reducers, testers
from pytorch_metric_learning.utils.accuracy_calculator import AccuracyCalculator

folder = '/content/drive/My Drive/PROJECTE DEEP LEARNING/TOP15_DATAS_WITH_100_PICTURES'
all_pictures = os.listdir(folder)

file_path = "/content/drive/My Drive/PROJECTE DEEP LEARNING/TOP15_DATAS_WITH_100_PICTURES.csv"
df = pd.read_csv(file_path)

# Reemplazar el prefijo "c." con nada en la columna 'date'
#df['date'] = df['date'].str.replace('c.', '').astype('int', errors='ignore')


print(df.head())

NUM_CLASSES = len(df['date'].unique().tolist())

print("Number of classes:", NUM_CLASSES)


                  artist  date      genre  pixelsx  pixelsy  size_bytes  \
0  Ernst Ludwig Kirchner  1913  landscape   4908.0   3684.0  10682834.0   
1          Koloman Moser  1913  landscape   4716.0   3420.0  10933062.0   
2          Koloman Moser  1913  landscape   4722.0   3102.0   9018058.0   
3       Ferdinand Hodler  1913   portrait   3156.0   4638.0   8453089.0   
4          Koloman Moser  1913     design   4734.0   2622.0   8317658.0   

    source                 style  \
0  wikiart         Expressionism   
1  wikiart             Symbolism   
2  wikiart             Symbolism   
3  wikiart  Art Nouveau (Modern)   
4  wikiart  Art Nouveau (Modern)   

                                            title    artist_group  in_train  \
0                           Red Tree on the Beach  train_and_test      True   
1                                 Mountain ranges  train_and_test      True   
2                          Landscape in Semmering  train_and_test      True   
3      Valentine

In [ ]:
import os
import shutil
import random
import pandas as pd


In [ ]:
from PIL import Image
Image.MAX_IMAGE_PIXELS = None

In [ ]:
# GROUP THE FILES BY EACH date
import random
def group_files_per_date():
  data_per_date = {}

  # Iterate over each row of the DataFrame
  for index, row in df.iterrows():
      date = row['date']
      new_filename = row['new_filename']

      # Check if the date is already in the dictionary
      if date in data_per_date:
          # Add the new_filename to the existing list of the date
          data_per_date[date].append(new_filename)
      else:
          # Create a new list for the date and add the new_filename
          data_per_date[date] = [new_filename]

  return data_per_date

# SPLIT THE FILES INTO TRAIN AND TEST SETS (CREATE NEW DATASETS)
def split_test_train(data_per_date, train_percentage = 0.8):

  TRAIN_names = []
  TEST_names = []

  # Iterate over each date and their 'new_filename'
  for date, filenames in data_per_date.items():
      # Calculate the number of files for training and testing
      total_files = len(filenames)
      num_train = int(train_percentage * total_files)

      # Shuffle the filenames to avoid selection biases
      random.shuffle(filenames)

      # Divide the filenames into train and test
      train_filenames = filenames[:num_train]
      test_filenames = filenames[num_train:]

      # Store the divided filenames into train and test
      TRAIN_names.extend(train_filenames)
      TEST_names.extend(test_filenames)
  return TRAIN_names, TEST_names



# CREATE TEST and TRAIN folders
def create_train_test_folders(folder, TRAIN_names, TEST_names):
    # Output folder for TRAIN and TEST
    train_folder = os.path.join(folder, 'TRAIN')
    test_folder = os.path.join(folder, 'TEST')

    # Create TRAIN and TEST folders if they don't exist
    os.makedirs(train_folder, exist_ok=True)
    os.makedirs(test_folder, exist_ok=True)

    # Move images from TRAIN to TRAIN folder
    for filename in TRAIN_names:
        src = os.path.join(folder, filename)
        dst = os.path.join(train_folder, filename)
        shutil.move(src, dst)

    # Move images from TEST to TEST folder
    for filename in TEST_names:
        src = os.path.join(folder, filename)
        dst = os.path.join(test_folder, filename)
        shutil.move(src, dst)

    # Create df_train with rows whose 'new_filename' is in TRAIN_names and keep only 'date' and 'new_filename' columns
    train_dataframe = df[df['new_filename'].isin(TRAIN_names)][['date', 'new_filename']]
    # File path for the CSV file
    file_path = os.path.join(folder, 'train_dataframe.csv')
    # Save the DataFrame as CSV
    train_dataframe.to_csv(file_path, index=False)

    # Create df_test with rows whose 'new_filename' is in TEST_names and keep only 'date' and 'new_filename' columns
    test_dataframe = df[df['new_filename'].isin(TEST_names)][['date', 'new_filename']]
    # File path for the CSV file
    file_path = os.path.join(folder, 'test_dataframe.csv')
    # Save the DataFrame as CSV
    test_dataframe.to_csv(file_path, index=False)

    return train_dataframe, test_dataframe

if not (os.path.exists(os.path.join(folder, 'TRAIN')) and os.path.exists(os.path.join(folder, 'TEST')) and os.path.exists(os.path.join(folder, 'train_dataframe.csv'))):
  # Group files per date
  data_per_date = group_files_per_date()
  # Print the number of files for each date
  print("- date and numer of images:")
  for key in data_per_date.keys():
    print(key, len(data_per_date[key]))

  # Split data
  TRAIN_names, TEST_names = split_test_train(data_per_date)

  # Use the function to create TRAIN and TEST folders
  train_dataframe, test_dataframe = create_train_test_folders(folder, TRAIN_names, TEST_names)
  print("\nImages successfully moved to TRAIN and TEST folders.")
else:
  print("TRAIN and TEST folders already exist. Reading train and test DataFrames from CSV files...")

  # Read train and test DataFrames from CSV files
  train_folder = os.path.join(folder, 'TRAIN')
  test_folder = os.path.join(folder, 'TEST')

  train_images = set(os.listdir(train_folder))
  test_images = set(os.listdir(test_folder))

  train_dataframe = df[df['new_filename'].isin(train_images)].copy()
  test_dataframe = df[df['new_filename'].isin(test_images)].copy()

  train_dataframe = train_dataframe[['new_filename', 'date']]
  test_dataframe = test_dataframe[['new_filename', 'date']]

  print("Train DataFrame:", train_dataframe.shape)
  print("Test DataFrame:", test_dataframe.shape)


TRAIN and TEST folders already exist. Reading train and test DataFrames from CSV files...
Train DataFrame: (1200, 2)
Test DataFrame: (300, 2)


In [ ]:
test_dataframe.groupby('date').size().reset_index(name='count')

,date,count
0,1885,20
1,1888,20
2,1889,20
3,1890,20
4,1895,20
5,1907,20
6,1908,20
7,1909,20
8,1910,20
9,1911,20


In [ ]:
num_valores_distintos = test_dataframe['date'].nunique()
print("Número de valores distintos en la columna 'date':", num_valores_distintos)


Número de valores distintos en la columna 'date': 15


# Model

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import models, transforms
from torch.utils.data import DataLoader

class ArtEmbeddingModel(nn.Module):
    def __init__(self):
        super(ArtEmbeddingModel, self).__init__()
        # Load pre-trained DenseNet model
        self.base_model = models.densenet121(pretrained=True)
        # Modify the classifier to output embeddings of size 128
        num_features = self.base_model.classifier.in_features
        self.base_model.classifier = nn.Linear(num_features, 128)

    def forward(self, x):
        # Forward pass through the base model
        features = self.base_model(x)
        return features

def triplet_loss(anchor, positive, negative, margin=1.0):
    distance_positive = (anchor - positive).pow(2).sum(1)  # Distancia L2
    distance_negative = (anchor - negative).pow(2).sum(1)  # Distancia L2
    losses = torch.relu(distance_positive - distance_negative + margin)
    return losses.mean()




# Triplets creation functions

In [ ]:
# Definir la clase TripletArtDataset
class TripletArtDataset(Dataset):
    def __init__(self, images, dates, similarity_threshold=5, transform=None):
        self.images = images
        self.dates = dates
        self.similarity_threshold = similarity_threshold
        self.transform = transform
        self.triplets = self.create_triplets()

    def create_triplets(self):
        triplets = []
        num_images = len(self.images)

        for i in range(num_images):
            anchor_image = self.images[i]
            anchor_date = self.dates[i]

            positive_indices = [j for j in range(num_images) if abs(self.dates[j] - anchor_date) <= self.similarity_threshold and j != i]
            negative_indices = [j for j in range(num_images) if abs(self.dates[j] - anchor_date) > self.similarity_threshold]

            if not positive_indices or not negative_indices:
                continue

            positive_index = np.random.choice(positive_indices)
            negative_index = np.random.choice(negative_indices)

            positive_image = self.images[positive_index]
            negative_image = self.images[negative_index]

            triplets.append((anchor_image, positive_image, negative_image))

        return triplets

    def __getitem__(self, index):
        anchor_path, positive_path, negative_path = self.triplets[index]

        # Imprimir las rutas para depuración
        print(f"Anchor path: {anchor_path}")
        print(f"Positive path: {positive_path}")
        print(f"Negative path: {negative_path}")

        try:
            anchor = Image.open(anchor_path)
            positive = Image.open(positive_path)
            negative = Image.open(negative_path)
        except FileNotFoundError as e:
            print(e)
            return None, None, None

        if self.transform:
            anchor = self.transform(anchor)
            positive = self.transform(positive)
            negative = self.transform(negative)

        return anchor, positive, negative

    def __len__(self):
        return len(self.triplets)

# Train and Test Dataloaders

In [ ]:
import os
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image
import numpy as np


# Asegúrate de que las rutas de las imágenes son completas
train_images = set(os.listdir(train_folder))
train_dataframe = df[df['new_filename'].isin(train_images)].copy()
train_dataframe = train_dataframe[['new_filename', 'date']]

# Crear una lista con las rutas completas de las imágenes
train_images_list = [os.path.join(train_folder, img) for img in train_images]

dates_train = []
for image_name in train_images:
    matching_row = train_dataframe[train_dataframe['new_filename'] == image_name]
    if not matching_row.empty:
        # Agregar la fecha correspondiente a la lista
        dates_train.append(matching_row.iloc[0]['date'])  # Asumimos que solo hay una fila que coincida
    else:
        # Manejo de casos donde no se encuentra la imagen
        dates_train.append(None)  # O cualquier otro valor indicativo de ausencia

# Convertir la lista filtrada de fechas a enteros
dates_train_int = [int(date) for date in dates_train]

print(dates_train_int)

# Definir las transformaciones
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])



# Crear datasets y dataloaders para entrenamiento
train_dataset = TripletArtDataset(train_images_list, dates_train_int, transform=transform)
train_dataloader = DataLoader(train_dataset, batch_size=32, shuffle=True)





[1914, 1911, 1888, 1908, 1908, 1885, 1911, 1915, 1909, 1913, 1915, 1885, 1913, 1889, 1917, 1889, 1888, 1907, 1907, 1895, 1908, 1913, 1909, 1912, 1889, 1909, 1895, 1889, 1907, 1888, 1914, 1917, 1890, 1889, 1895, 1890, 1915, 1913, 1911, 1889, 1911, 1917, 1913, 1913, 1889, 1917, 1912, 1917, 1890, 1917, 1915, 1895, 1913, 1909, 1914, 1915, 1888, 1914, 1909, 1915, 1889, 1911, 1912, 1912, 1910, 1908, 1889, 1885, 1908, 1890, 1914, 1889, 1889, 1908, 1908, 1908, 1908, 1890, 1890, 1888, 1911, 1911, 1908, 1911, 1889, 1913, 1908, 1889, 1907, 1912, 1888, 1885, 1912, 1917, 1912, 1909, 1889, 1895, 1912, 1895, 1910, 1907, 1915, 1909, 1910, 1889, 1912, 1889, 1913, 1915, 1888, 1913, 1913, 1895, 1915, 1890, 1890, 1909, 1890, 1909, 1885, 1915, 1889, 1913, 1912, 1888, 1908, 1888, 1910, 1915, 1914, 1911, 1895, 1895, 1908, 1909, 1915, 1911, 1917, 1913, 1907, 1885, 1911, 1914, 1890, 1914, 1888, 1885, 1889, 1911, 1907, 1911, 1909, 1907, 1913, 1912, 1907, 1890, 1890, 1885, 1909, 1911, 1912, 1917, 1910, 1913, 191

In [ ]:
# Asegúrate de que las rutas de las imágenes son completas
test_images = set(os.listdir(test_folder))
test_dataframe = df[df['new_filename'].isin(test_images)].copy()
test_dataframe = test_dataframe[['new_filename', 'date']]

# Crear una lista con las rutas completas de las imágenes
test_images_list = [os.path.join(test_folder, img) for img in test_images]

dates_test = []
for image_name in test_images:
    matching_row = test_dataframe[test_dataframe['new_filename'] == image_name]
    if not matching_row.empty:
        # Agregar la fecha correspondiente a la lista
        dates_test.append(matching_row.iloc[0]['date'])  # Asumimos que solo hay una fila que coincida
    else:
        # Manejo de casos donde no se encuentra la imagen
        dates_test.append(None)  # O cualquier otro valor indicativo de ausencia

# Convertir la lista filtrada de fechas a enteros
dates_test_int = [int(date) for date in dates_test]

print(dates_test_int)

# Definir las transformaciones
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])



# Crear datasets y dataloaders para entrenamiento
test_dataset = TripletArtDataset(test_images_list, dates_test_int, transform=transform)
test_dataloader = DataLoader(test_dataset, batch_size=32, shuffle=True)

[1885, 1912, 1911, 1909, 1890, 1917, 1889, 1907, 1911, 1889, 1915, 1911, 1912, 1914, 1888, 1914, 1915, 1895, 1914, 1908, 1911, 1895, 1915, 1915, 1885, 1908, 1913, 1914, 1913, 1895, 1910, 1913, 1913, 1889, 1890, 1912, 1912, 1912, 1890, 1908, 1909, 1911, 1890, 1907, 1913, 1917, 1888, 1915, 1895, 1909, 1890, 1910, 1913, 1912, 1909, 1915, 1890, 1910, 1895, 1908, 1888, 1913, 1885, 1912, 1889, 1915, 1885, 1907, 1915, 1915, 1885, 1915, 1889, 1913, 1889, 1908, 1890, 1909, 1910, 1890, 1915, 1912, 1888, 1885, 1909, 1910, 1914, 1907, 1885, 1908, 1885, 1914, 1917, 1907, 1915, 1888, 1913, 1917, 1908, 1914, 1910, 1909, 1910, 1909, 1888, 1914, 1914, 1907, 1912, 1885, 1917, 1895, 1914, 1907, 1885, 1917, 1895, 1910, 1909, 1889, 1890, 1907, 1914, 1885, 1910, 1915, 1888, 1889, 1895, 1888, 1912, 1907, 1908, 1909, 1889, 1890, 1907, 1910, 1915, 1912, 1914, 1907, 1917, 1911, 1914, 1911, 1889, 1907, 1917, 1907, 1895, 1913, 1889, 1910, 1915, 1908, 1910, 1885, 1888, 1890, 1913, 1911, 1911, 1889, 1890, 1908, 190

# ENTRENAMENT

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = ArtEmbeddingModel().to(device)
optimizer = optim.Adam(model.parameters(), lr=0.01)
num_epochs = 25

for epoch in range(num_epochs):
    model.train()
    for anchors, positives, negatives in train_dataloader:
        anchors = anchors.to(device)
        positives = positives.to(device)
        negatives = negatives.to(device)

        # Imprimir las rutas para depuración
        print(f"Anchor path: {anchors}")
        print(f"Positive path: {positives}")
        print(f"Negative path: {negatives}")

        optimizer.zero_grad()
        anchor_embeddings = model(anchors)
        positive_embeddings = model(positives)
        negative_embeddings = model(negatives)

        loss = triplet_loss(anchor_embeddings, positive_embeddings, negative_embeddings)
        loss.backward()
        optimizer.step()

    print(f"Epoch {epoch+1}, Loss: {loss.item()}")

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=DenseNet121_Weights.IMAGENET1K_V1`. You can also use `weights=DenseNet121_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/densenet121-a639ec97.pth" to /root/.cache/torch/hub/checkpoints/densenet121-a639ec97.pth
100%|██████████| 30.8M/30.8M [00:14<00:00, 2.17MB/s]


Se han truncado las últimas 5000 líneas del flujo de salida.


        ...,


        [[[0.9098, 0.8627, 0.8824,  ..., 0.2118, 0.2353, 0.2549],
          [0.4627, 0.4078, 0.4078,  ..., 0.2353, 0.2275, 0.2039],
          [0.2824, 0.2196, 0.2039,  ..., 0.4941, 0.4353, 0.3725],
          ...,
          [0.3686, 0.2706, 0.2784,  ..., 0.9412, 0.9294, 0.9176],
          [0.6000, 0.5294, 0.5647,  ..., 0.9490, 0.9412, 0.9333],
          [1.0000, 0.9569, 0.9922,  ..., 0.9294, 0.9294, 0.9255]],

         [[0.9412, 0.8941, 0.8941,  ..., 0.1529, 0.1765, 0.2000],
          [0.4784, 0.4235, 0.4157,  ..., 0.1882, 0.1686, 0.1569],
          [0.2824, 0.2196, 0.2039,  ..., 0.4510, 0.3882, 0.3333],
          ...,
          [0.3176, 0.2196, 0.2275,  ..., 0.9294, 0.9176, 0.9059],
          [0.5373, 0.4706, 0.5059,  ..., 0.9647, 0.9529, 0.9490],
          [0.9451, 0.8863, 0.9216,  ..., 0.9490, 0.9490, 0.9451]],

         [[0.9490, 0.9020, 0.9137,  ..., 0.1647, 0.1882, 0.2000],
          [0.5137, 0.4588, 0.4

# EVALUATE MODEL

In [ ]:
def generate_embeddings(dataloader, model, device):
    model.eval()
    embeddings = []
    labels = []
    with torch.no_grad():
        for images, _, _ in dataloader:
            images = images.to(device)
            emb = model(images)
            embeddings.append(emb.cpu().numpy())

    embeddings = np.concatenate(embeddings)
    return embeddings

# Generar embeddings para el conjunto de entrenamiento
train_embeddings = generate_embeddings(train_dataloader, model, device)
train_labels = dates_train_int  # Usamos las fechas como etiquetas de referencia

# Asegúrate de que las longitudes de embeddings y etiquetas coinciden
assert len(train_embeddings) == len(train_labels), "La cantidad de embeddings y etiquetas no coincide."


Anchor path: /content/drive/My Drive/PROJECTE DEEP LEARNING/TOP15_DATAS_WITH_100_PICTURES/TRAIN/10805.jpg
Positive path: /content/drive/My Drive/PROJECTE DEEP LEARNING/TOP15_DATAS_WITH_100_PICTURES/TRAIN/74438.jpg
Negative path: /content/drive/My Drive/PROJECTE DEEP LEARNING/TOP15_DATAS_WITH_100_PICTURES/TRAIN/48930.jpg
Anchor path: /content/drive/My Drive/PROJECTE DEEP LEARNING/TOP15_DATAS_WITH_100_PICTURES/TRAIN/68177.jpg
Positive path: /content/drive/My Drive/PROJECTE DEEP LEARNING/TOP15_DATAS_WITH_100_PICTURES/TRAIN/55263.jpg
Negative path: /content/drive/My Drive/PROJECTE DEEP LEARNING/TOP15_DATAS_WITH_100_PICTURES/TRAIN/69422.jpg
Anchor path: /content/drive/My Drive/PROJECTE DEEP LEARNING/TOP15_DATAS_WITH_100_PICTURES/TRAIN/72922.jpg
Positive path: /content/drive/My Drive/PROJECTE DEEP LEARNING/TOP15_DATAS_WITH_100_PICTURES/TRAIN/35914.jpg
Negative path: /content/drive/My Drive/PROJECTE DEEP LEARNING/TOP15_DATAS_WITH_100_PICTURES/TRAIN/70196.jpg
Anchor path: /content/drive/My Dri

In [ ]:
# Generar embeddings para el conjunto de prueba
test_embeddings = generate_embeddings(test_dataloader, model, device)
test_labels = dates_test_int  # Usamos las fechas como etiquetas de prueba (esto es solo para verificar)


Anchor path: /content/drive/My Drive/PROJECTE DEEP LEARNING/TOP15_DATAS_WITH_100_PICTURES/TEST/8774.jpg
Positive path: /content/drive/My Drive/PROJECTE DEEP LEARNING/TOP15_DATAS_WITH_100_PICTURES/TEST/32927.jpg
Negative path: /content/drive/My Drive/PROJECTE DEEP LEARNING/TOP15_DATAS_WITH_100_PICTURES/TEST/85069.jpg
Anchor path: /content/drive/My Drive/PROJECTE DEEP LEARNING/TOP15_DATAS_WITH_100_PICTURES/TEST/51214.jpg
Positive path: /content/drive/My Drive/PROJECTE DEEP LEARNING/TOP15_DATAS_WITH_100_PICTURES/TEST/66642.jpg
Negative path: /content/drive/My Drive/PROJECTE DEEP LEARNING/TOP15_DATAS_WITH_100_PICTURES/TEST/3965.jpg
Anchor path: /content/drive/My Drive/PROJECTE DEEP LEARNING/TOP15_DATAS_WITH_100_PICTURES/TEST/79281.jpg
Positive path: /content/drive/My Drive/PROJECTE DEEP LEARNING/TOP15_DATAS_WITH_100_PICTURES/TEST/583.jpg
Negative path: /content/drive/My Drive/PROJECTE DEEP LEARNING/TOP15_DATAS_WITH_100_PICTURES/TEST/49056.jpg
Anchor path: /content/drive/My Drive/PROJECTE D

In [ ]:
# Ruta de la carpeta de prueba
test_folder = folder + "/TEST"

# Obtener todos los nombres de archivo dentro de la carpeta de prueba
test_filenames = os.listdir(test_folder)

# Crear la lista completa de rutas de archivo para las imágenes de prueba
test_filenames = [os.path.join(test_folder, filename) for filename in test_filenames]

In [ ]:
# Generar embeddings para el conjunto de prueba
test_embeddings = generate_embeddings(test_dataloader, model, device)
test_labels = dates_test_int  # Usamos las fechas como etiquetas de prueba (esto es solo para verificar)

# Crear un diccionario que mapee los nombres de archivo con las etiquetas reales
test_labels_dict = dict(zip(test_filenames, test_labels))




Anchor path: /content/drive/My Drive/PROJECTE DEEP LEARNING/TOP15_DATAS_WITH_100_PICTURES/TEST/102420.jpg
Positive path: /content/drive/My Drive/PROJECTE DEEP LEARNING/TOP15_DATAS_WITH_100_PICTURES/TEST/7771.jpg
Negative path: /content/drive/My Drive/PROJECTE DEEP LEARNING/TOP15_DATAS_WITH_100_PICTURES/TEST/52702.jpg
Anchor path: /content/drive/My Drive/PROJECTE DEEP LEARNING/TOP15_DATAS_WITH_100_PICTURES/TEST/66096.jpg
Positive path: /content/drive/My Drive/PROJECTE DEEP LEARNING/TOP15_DATAS_WITH_100_PICTURES/TEST/4929.jpg
Negative path: /content/drive/My Drive/PROJECTE DEEP LEARNING/TOP15_DATAS_WITH_100_PICTURES/TEST/14659.jpg
Anchor path: /content/drive/My Drive/PROJECTE DEEP LEARNING/TOP15_DATAS_WITH_100_PICTURES/TEST/62629.jpg
Positive path: /content/drive/My Drive/PROJECTE DEEP LEARNING/TOP15_DATAS_WITH_100_PICTURES/TEST/14378.jpg
Negative path: /content/drive/My Drive/PROJECTE DEEP LEARNING/TOP15_DATAS_WITH_100_PICTURES/TEST/86923.jpg
Anchor path: /content/drive/My Drive/PROJECT

In [ ]:
# Entrenar el clasificador k-NN usando los embeddings de entrenamiento
knn = KNeighborsClassifier(n_neighbors=1)
knn.fit(train_embeddings, train_labels)

# Predecir las clases para los embeddings de prueba
predicted_labels = knn.predict(test_embeddings)

real_labels = []
# Imprimir los resultados junto con los nombres de archivo y las etiquetas reales
for i, (filename, label) in enumerate(zip(test_filenames, predicted_labels)):
    filename = filename.split("/")[-1]
    real_label = test_dataframe[test_dataframe["new_filename"] == filename]['date'].values[0]
    real_labels.append(real_label)
    print(f"La imagen {filename} pertenece a la clase: {label}. Etiqueta real: {real_label}")

In [ ]:
from collections import Counter

# Obtener el conteo de valores
label_counts = Counter(predicted_labels)

# Imprimir los conteos
for label, count in label_counts.items():
    print(f"Clase {label} tiene {count} instancias predichas.")


Clase 1914 tiene 17 instancias predichas.
Clase 1915 tiene 23 instancias predichas.
Clase 1911 tiene 35 instancias predichas.
Clase 1895 tiene 15 instancias predichas.
Clase 1890 tiene 25 instancias predichas.
Clase 1910 tiene 25 instancias predichas.
Clase 1888 tiene 18 instancias predichas.
Clase 1909 tiene 19 instancias predichas.
Clase 1913 tiene 18 instancias predichas.
Clase 1907 tiene 14 instancias predichas.
Clase 1908 tiene 22 instancias predichas.
Clase 1912 tiene 17 instancias predichas.
Clase 1889 tiene 17 instancias predichas.
Clase 1885 tiene 14 instancias predichas.
Clase 1917 tiene 21 instancias predichas.


In [ ]:
import numpy as np


# Convertir las listas a arrays de NumPy para facilitar los cálculos
predicted_labels_arr = np.array(predicted_labels)
real_labels_arr = np.array(real_labels)

# Calcular el Error Cuadrático Medio (MSE)
mse = np.mean((predicted_labels_arr - real_labels_arr) ** 2)

print(f"El Error Cuadrático Medio (MSE) entre predicted_labels y real_labels es: {mse}")


KeyboardInterrupt: 